In [1]:
import numpy as np
import pandas as pd 
from pandas_profiling import ProfileReport 

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

all_data = pd.read_csv('./Resources/allData.csv', low_memory=False) 

#changing coLumn names for easier use 
all_data = all_data.rename(columns=str.lower) 
all_data = all_data.rename(columns=str.strip) 
all_data.columns = all_data.columns.str.replace(" ", "_")

#Creating a Chicago dataframe with reLevant attributes 
df = all_data[all_data['city'] == 'CHICAGO'][['id', 'legal_name', 'doing_business_as_name', 'state', 'city', 
                                              'zip_code', 'address', 'ward', 'precinct', 'ward_precinct', 'police_district', 'latitude', 'longitude', 'location', 
                                              'business_activity', 'business_activity_id']] 

df = df.where(pd.notnull(df), None)

In [2]:
'''Using profile report to extract the number of missing values for the initial dataset
   
   prints a list of attributes with corresponding missing values
   
   uses pandas_profiling's class Profile report
'''
report = ProfileReport(df)
report = report.get_description()

print('Dataset length: ' + str(len(df)))
for key in report['variables'].keys():
    print(key + ": " + str(report['variables'][key]['n_missing']))
    
#addresses have dummy values
#instead of an address [REDACTED FOR PRIVACY] is written
print('Count of address dummy values: ' + str(len(df[df['address'] == '[REDACTED FOR PRIVACY]'])))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Dataset length: 51505
id: 0
legal_name: 0
doing_business_as_name: 36
state: 0
city: 0
zip_code: 12
address: 0
ward: 61
precinct: 12530
ward_precinct: 61
police_district: 12091
latitude: 954
longitude: 954
location: 954
business_activity: 4312
business_activity_id: 4312
Count of address dummy values: 302


In [3]:
'''Formatting business_activitiy and business_activity_id

   converting a string of values to list
   
   using functions split_activity and split_activity_int
'''
def split_activity(activities):
    '''takes a string of activities connected with |
       returns a list of strings, splitted on |
    '''
    if activities != None:
        return activities.split(" | ")
    
def split_activity_int(activities):
    '''takes a string of numerical activity ids connected with |
       returns a list of integers, splitted on |
    '''
    if activities != None:
        temp = activities.split(" | ")
        return [int(x) for x in temp]    
    
#splitting every cell of activities and activity ids, so I can get an array for businesses with more than one activity
df['business_activities'] = df['business_activity'].apply(lambda x: split_activity(x))
df['business_activities_ids'] = df['business_activity_id'].apply(split_activity_int)

In [4]:
'''Merging the businesses that appear several times in df

   - creating new attributes business_activities and business_activities_ids
   - converting strings business_activity and business_activity_id and placing the values to newly created attributes
   - removes duplicate values from business_activities and business_activities_ids
   
   using split_activity, split_activity_int, string_to_list
'''
def split_activity(activities):
    '''takes a string of activities connected with |
       returns a list of strings, splitted on |
    '''
    if activities != None:
        return activities.split(" | ")
    
def split_activity_int(activities):
    '''takes a string of numerical activity ids connected with |
       returns a list of integers, splitted on |
    '''
    if activities != None:
        temp = activities.split(" | ")
        return [int(x) for x in temp] 
    
def remove_duplicates(df_series_object):
    '''business_activities and business_activities_ids contained duplicate values
    in the cases of no values, 0 was present instead of []
    This function removes duplicate values and replaces 0 with []'''
    if df_series_object == 0:
        return []
    return [*set(df_series_object)]
    
def string_to_list(text):
    temp = []
    temp.append(text)
    return temp

df['business_activities'] = df['business_activity'].apply(split_activity)
df['business_activities_ids'] = df['business_activity_id'].apply(split_activity_int)
df['id'] = df['id'].apply(string_to_list)

df = (df.groupby(['legal_name', 'doing_business_as_name', 'address', 'ward', 'precinct', 'ward_precinct', 'police_district', 'state', 'city', 'zip_code', 'latitude', 'longitude', 'location'], dropna = False)[['id','business_activities', 'business_activities_ids']].sum().reset_index())

#taking only the first id
df['id'] = df['id'].apply(lambda x: str(x[0]))

df['business_activities'] = df['business_activities'].apply(remove_duplicates)
df['business_activities_ids'] = df['business_activities_ids'].apply(remove_duplicates)

In [5]:
'''Using profile report to extract the number of missing values for the modified dataset
   
   prints a list of attributes with corresponding missing values
   
   uses pandas_profiling's class Profile report
'''
report = ProfileReport(df)
report = report.get_description()

print('Dataset length: ' + str(len(df)))
for key in report['variables'].keys():
    print(key + ": " + str(report['variables'][key]['n_missing']))
    
#addresses have dummy values
#instead of an address [REDACTED FOR PRIVACY] is written
print('Count of address dummy values: ' + str(len(df[df['address'] == '[REDACTED FOR PRIVACY]'])))

#checking for unique IDs
print("Unique IDs:" + str(report['variables']['id']['n_unique']))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Dataset length: 41399
legal_name: 0
doing_business_as_name: 36
address: 0
ward: 55
precinct: 11345
ward_precinct: 55
police_district: 11067
state: 0
city: 0
zip_code: 12
latitude: 687
longitude: 687
location: 687
id: 0
business_activities: 0
business_activities_ids: 0
Count of address dummy values: 246
Unique IDs:41399


In [6]:
'''Business-activity standardization

   loads .csv with business activities and their standardized values
   makes a name_stand_dict with the key:value being a business activity and its standardized value
   adds an attribute standardized_activities to the df based on names_stand_dict
   
   uses function standardize to create standardized_activities attribute
'''
activities_stand = pd.read_csv('./Resources/standardizedActivities.csv')

activities_names = activities_stand['original_value']
activities_stand_values = activities_stand['standardized_value']

activities_stand_values = activities_stand_values.to_list()

names_stand_dict = {}
for i in range(len(activities_names)):
    names_stand_dict[activities_names[i]] = activities_stand_values[i]
    
def standardize(dict_object, dframe_object, series_name):
    #dict_objec{series_name value: standardized value}
    #replaces series_name values with the ones from dict_object
    temp = dframe_object.copy()[series_name]
    temp = temp.apply(lambda x: [dict_object[i] for i in x if i in dict_object.keys()])
    return temp

df['standardized_activities'] = standardize(names_stand_dict, df, 'business_activities')

In [7]:
#Checking whether every recod has Chicago as the city, and Illinois as the state
print("City: Chicago, State Illinois: " + str(len(df[(df['city']=='CHICAGO') & (df['state']=="IL")])))
print("City: Chicago, State not Illinois: " + str(len(df[(df['city']=='CHICAGO') & (df['state']!="IL")])))

#Checking if every record has Chicago zip code (start with 606, 607, or 608)
print("Number of Chicago ZIP codes: " + str(len(df[df['zip_code'].apply(lambda x: str(x)[:3] in ['606', '607', '608'])])))
print("Number of non-Chicago ZIP codes: " + str(len(df[df['zip_code'].apply(lambda x: str(x)[:3] not in ['606', '607', '608'])])))

City: Chicago, State Illinois: 41397
City: Chicago, State not Illinois: 2
Number of Chicago ZIP codes: 41362
Number of non-Chicago ZIP codes: 37


In [8]:
#Two records that have Chicago as city, but the state is not Illinois
df[(df['city']=='CHICAGO') & (df['state']!="IL")][['id', 'city', 'state']]

,id,city,state
18416,2652776-20210316,CHICAGO,ID
35750,2627344-20211016,CHICAGO,IN


In [9]:
'''Creating a dataframe with high quality data (has legal name, and DBA, address, state, 
   Chicago ZIP code, and list of business activities)
   
   uses get_high_quality_data function'''

def get_high_quality_data(dframe):
    """returns a dataframe containing only records that meet the following criteria: 
            name: it must have both legal name and DBA. 
            address: it must have address and location. [REDACTED FOR PRIVACY] not allowed
            state: it must be in Illinois
            ZIP Code: Chicago ZIP codes start with 606, 607, or 608
            business activities: it has to contain the list of business activities
    """
    copy_df = dframe.copy(deep=True)
    
    copy_df['business_activities'] = copy_df['business_activities'].apply(lambda x: tuple(x))
    copy_df['business_activities_ids'] = copy_df['business_activities_ids'].apply(lambda x: tuple(x))
    copy_df['standardized_activities'] = copy_df['standardized_activities'].apply(lambda x: tuple(x))

    high_quality = copy_df[(copy_df['doing_business_as_name'].notnull()) 
                          & (copy_df['address'] != '[REDACTED FOR PRIVACY]') 
                          & (copy_df['state'] == 'IL')
                          & (copy_df['zip_code'].apply(lambda x: str(x)[:3] in ['606', '607', '608']))
                          & (copy_df['location'].notnull()) 
                          & (copy_df['business_activities'].apply(lambda x: len(x) != 0))]
    
    return high_quality

high_quality = get_high_quality_data(df)

In [10]:
print("Percentage of high-quality data: " + str(round(len(high_quality)/len(df)*100, 0)) + "%")

Percentage of high-quality data: 88.0%
